In [1]:
import pandas as pd



In [2]:
names = ["id","auteur","lien_auteur","nom_auteur","reclamation_examinee","date_examination","source","lien","pub_nom_auteur","pub_lien_auteur",
         "date_publication","contenu","extra_auteur","extra_contenu","extra_publication_examinée",
         "extra_mot_cle","extra_lien_reference","extra_tags","extra_titre","note_nom_alternatif","meilleure_note",
         "valeur_note","pire_note"]
df = pd.read_csv('../Dataset/claim.csv', sep=',', names = names, skiprows = 1, nrows = 5000)
df2 = pd.read_csv('../Dataset/claim.csv', sep=',', names = names, skiprows = 5001, nrows = 5000)
df3 = pd.read_csv('../Dataset/claim.csv', sep=',', names = names, skiprows = 10001, nrows = 5000)
df4 = pd.read_csv('../Dataset/claim.csv', sep=',', names = names, skiprows = 15001, nrows = 5000)




In [3]:
df = pd.concat([df, df2])
df = pd.concat([df, df3])
df = pd.concat([df, df4])
display(df.sample(5))


,id,auteur,lien_auteur,nom_auteur,reclamation_examinee,date_examination,source,lien,pub_nom_auteur,pub_lien_auteur,...,extra_contenu,extra_publication_examinée,extra_mot_cle,extra_lien_reference,extra_tags,extra_titre,note_nom_alternatif,meilleure_note,valeur_note,pire_note
3976,8976,NaN,snopes,NaN,A casting call for a Latina actress to play a ...,2016-03-08,snopes,https://www.snopes.com/fact-check/donald-trump...,NaN,NaN,...,"[{""id"" : 7529378"",""""begin"": 74,""end"": 82,""enti...","[{""id"" : 4848272"",""""begin"": 46,""end"": 58,""enti...","[{""id"" : 4848272"",""""begin"": 13,""end"": 25,""enti...",https://www.facebook.com/photo.php?fbid=101534...,"casting call, donald trump, trump actor",Donald Trump Isn’t Hiring a Latina Actress to ...,Mixture,NaN,NaN,NaN
3812,8812,NaN,snopes,NaN,"A video featuring the """"Best Volleyball Blocks...",2016-03-29,snopes,https://www.snopes.com/fact-check/volleyball-b...,NaN,NaN,...,"[{""id"" : 32558"",""""begin"": 293,""end"": 303,""enti...","[{""id"" : 32558"",""""begin"": 27,""end"": 37,""entity...","[{""id"" : 49991644"",""""begin"": 0,""end"": 14,""enti...",NaN,"scott sterling, studio c",‘Best Volleyball Blocks Ever’,False,NaN,NaN,NaN
3616,13616,NaN,snopes,NaN,Italian wedding soup gained its name from bein...,2013-04-16,snopes,https://www.snopes.com/fact-check/italian-wedd...,NaN,NaN,...,"[{""id"" : 531674"",""""begin"": 6,""end"": 26,""entity...","[{""id"" : 531674"",""""begin"": 0,""end"": 20,""entity...",[],NaN,"ASP Article, kiwi banana",Italian Wedding Soup Origins,FALSE,NaN,NaN,NaN
4513,4513,NaN,snopes,NaN,"TV host Bill Nye """"the Science Guy"""" was arres...",2019-09-12,snopes,https://www.snopes.com/fact-check/bill-nye-arr...,NaN,NaN,...,"[{""id"" : 717037"",""""begin"": 90,""end"": 114,""enti...","[{""id"" : 717037"",""""begin"": 8,""end"": 32,""entity...",[],https://www.snopes.com/tachyon/2019/09/nye-twe...,NaN,Was TV Host Bill Nye Arrested for Manufacturin...,Labeled Satire,NaN,NaN,NaN
3753,13753,NaN,snopes,NaN,Family of abortion clinic owner dies when plan...,2013-01-09,snopes,https://www.snopes.com/fact-check/feldkamp-pla...,NaN,NaN,...,"[{""id"" : 765"",""""begin"": 16,""end"": 24,""entity"":...","[{""id"" : 765"",""""begin"": 10,""end"": 18,""entity"":...",[],"http://www.fpamg.net/,http://www.findagrave.co...",ASP Article,Feldkamp Plane Crash,MIXTURE,NaN,NaN,NaN


In [4]:
display(df.shape)

(20000, 23)

In [5]:
display(df.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20000 entries, 0 to 4999
Data columns (total 23 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   id                          20000 non-null  int64  
 1   auteur                      0 non-null      float64
 2   lien_auteur                 20000 non-null  object 
 3   nom_auteur                  0 non-null      float64
 4   reclamation_examinee        20000 non-null  object 
 5   date_examination            19598 non-null  object 
 6   source                      20000 non-null  object 
 7   lien                        20000 non-null  object 
 8   pub_nom_auteur              1525 non-null   object 
 9   pub_lien_auteur             113 non-null    object 
 10  date_publication            1525 non-null   object 
 11  contenu                     19997 non-null  object 
 12  extra_auteur                20000 non-null  object 
 13  extra_contenu               2000

None

On supprime toutes les colonnes qui ou toutes les lignes sont Nan 

In [6]:
del df["auteur"]
del df["nom_auteur"]
del df["pub_nom_auteur"]
del df["pub_lien_auteur"]
del df["date_publication"]
del df["meilleure_note"]
del df["valeur_note"]
del df["pire_note"]

In [10]:
display(df.sample(5))

,id,lien_auteur,reclamation_examinee,date_examination,source,lien,contenu,extra_auteur,extra_contenu,extra_publication_examinée,extra_mot_cle,extra_lien_reference,extra_tags,extra_titre,note_nom_alternatif
2573,7573,snopes,Republican nominee Donald Trump called Preside...,2016-09-01,snopes,https://www.snopes.com/fact-check/donald-trump...,It is hardly a secret that Republican presiden...,[],"[{""id"" : 32070"",""""begin"": 27,""end"": 37,""entity...","[{""id"" : 772738"",""""begin"": 0,""end"": 10,""entity...","[{""id"" : 21377251"",""""begin"": 0,""end"": 13,""enti...","https://www.youtube.com/watch?v=M6DSlUCe__4,ht...","2016 election, barack obama, donald trump, ele...",Donald Trump Called President Obama a ‘Lying N...,False
3925,450,politifact,"""""In two years, Gov. McAuliffe has vetoed more...",2019-10-14,politifact,http://www.politifact.com/virginia/statements/...,Gov. Terry McAuliffe is way too heavy-handed w...,"[{""id"" : 20283924"",""""begin"": 0,""end"": 8,""entit...","[{""id"" : 17878405"",""""begin"": 4,""end"": 19,""enti...",[],[],http://bluevirginia.us/2016/04/video-va-house-...,Voting Record,Kirk Cox says McAuliffe has vetoed more bills ...,True
2351,2351,snopes,A photograph documents a scary moment when a b...,2018-07-12,snopes,https://www.snopes.com/fact-check/bear-attack-...,Images purportedly showing the moment before a...,[],"[{""id"" : 17727"",""""begin"": 135,""end"": 147,""enti...",[],"[{""id"" : 43355202"",""""begin"": 0,""end"": 11,""enti...",https://www.snopes.com/fact-check/self-destruc...,"designcrowd, worth1000",Was This Photograph Taken Moments Before a Bea...,False
1718,6718,snopes,"TV host Bill Nye """"the Science Guy"""" was arres...",2019-09-12,snopes,https://www.snopes.com/fact-check/bill-nye-arr...,"In September 2019, a years-old rumor started s...",[],"[{""id"" : 717037"",""""begin"": 90,""end"": 114,""enti...","[{""id"" : 717037"",""""begin"": 8,""end"": 32,""entity...",[],https://www.snopes.com/tachyon/2019/09/nye-twe...,NaN,Was TV Host Bill Nye Arrested for Manufacturin...,Labeled Satire
2599,2599,snopes,State Farm insurance will not offer insurance ...,2018-05-31,snopes,https://www.snopes.com/fact-check/is-state-far...,Though some gun owners have accused State Farm...,[],"[{""id"" : 810859"",""""begin"": 36,""end"": 56,""entit...","[{""id"" : 810859"",""""begin"": 0,""end"": 20,""entity...","[{""id"" : 450957"",""""begin"": 0,""end"": 10,""entity...",https://www.snopes.com/tachyon/2018/05/State-F...,"gun rights, guns, nra, state farm",Is State Farm Denying Insurance to Gun Owners?,Mostly False
4875,14875,snopes,NASA and NOAA faked climate data in the GISTEM...,2019-10-03,snopes,https://www.snopes.com/fact-check/global-warmi...,"Since 2014, climate-change deniers have squeez...",[],"[{""id"" : 7529378"",""""begin"": 65,""end"": 73,""enti...","[{""id"" : 18426568"",""""begin"": 0,""end"": 4,""entit...",[],https://www.buzzfeednews.com/article/stephanie...,NaN,Was Global Warming Data ‘Faked’ to ‘Fit Climat...,False
737,5737,snopes,President Donald Trump cut funding for the vet...,2017-03-13,snopes,https://www.snopes.com/fact-check/veterans-sui...,"On 11 March 2017, the web site The Satira Trib...",[],"[{""id"" : 4848272"",""""begin"": 95,""end"": 107,""ent...","[{""id"" : 4848272"",""""begin"": 10,""end"": 22,""enti...","[{""id"" : 4848272"",""""begin"": 0,""end"": 12,""entit...",http://www.snopes.com/2016/01/14/fake-news-sit...,"donald trump, satira tribune",Trump Cuts Veteran Suicide Hotline Funding?,False
2701,12701,snopes,"TV host Bill Nye """"the Science Guy"""" was arres...",2019-09-12,snopes,https://www.snopes.com/fact-check/bill-nye-arr...,"In September 2019, a years-old rumor started s...",[],"[{""id"" : 717037"",""""begin"": 90,""end"": 114,""enti...","[{""id"" : 717037"",""""begin"": 8,""end"": 32,""entity...",[],https://www.snopes.com/tachyon/2019/09/nye-twe...,NaN,Was TV Host Bill Nye Arrested for Manufacturin...,Labeled Satire
895,15895,snopes,Car thieves at the Deerbrook Mal

Nouveau nombre de lignes et colonnes 

In [8]:
display(df.shape)

(20000, 15)

In [9]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20000 entries, 0 to 4999
Data columns (total 15 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   id                          20000 non-null  int64 
 1   lien_auteur                 20000 non-null  object
 2   reclamation_examinee        20000 non-null  object
 3   date_examination            19598 non-null  object
 4   source                      20000 non-null  object
 5   lien                        20000 non-null  object
 6   contenu                     19997 non-null  object
 7   extra_auteur                20000 non-null  object
 8   extra_contenu               20000 non-null  object
 9   extra_publication_examinée  20000 non-null  object
 10  extra_mot_cle               20000 non-null  object
 11  extra_lien_reference        18632 non-null  object
 12  extra_tags                  14175 non-null  object
 13  extra_titre                 20000 non-null  obj

In [12]:
display(df["extra_auteur"].describe())

count     20000
unique      657
top          []
freq      18696
Name: extra_auteur, dtype: object

In [13]:
del df["extra_auteur"]

In [16]:
for column in df :
    print(column)
    display(df[column].describe())
    print("___________________")
    

id


count    20000.000000
mean      8590.781250
std       5599.143886
min          0.000000
25%       3474.750000
50%       8474.500000
75%      13474.250000
max      18474.000000
Name: id, dtype: float64

___________________
lien_auteur


count      20000
unique         2
top       snopes
freq       18475
Name: lien_auteur, dtype: object

___________________
reclamation_examinee


...

___________________
extra_titre


count                                                 20000
unique                                                14279
top       Was Global Warming Data ‘Faked’ to ‘Fit Climat...
freq                                                   1132
Name: extra_titre, dtype: object

___________________
note_nom_alternatif


count     19992
unique      128
top       False
freq       7250
Name: note_nom_alternatif, dtype: object

___________________


In [20]:
df.drop_duplicates(subset=["lien_auteur","contenu"],keep='first', inplace=True)

In [21]:
for column in df :
    print(column)
    display(df[column].describe())
    print("___________________")

id


count    14318.000000
mean      8287.070610
std       5687.421288
min          0.000000
25%       2911.250000
50%       7990.500000
75%      13291.750000
max      18454.000000
Name: id, dtype: float64

___________________
lien_auteur


count      14318
unique         2
top       snopes
freq       12796
Name: lien_auteur, dtype: object

___________________
reclamation_examinee


...

___________________
extra_titre


count                       14318
unique                      14276
top       Adam Sandler Death Hoax
freq                            3
Name: extra_titre, dtype: object

___________________
note_nom_alternatif


count     14311
unique      128
top       False
freq       4979
Name: note_nom_alternatif, dtype: object

___________________


In [22]:
display(df.sample(10))

,id,lien_auteur,reclamation_examinee,date_examination,source,lien,contenu,extra_contenu,extra_publication_examinée,extra_mot_cle,extra_lien_reference,extra_tags,extra_titre,note_nom_alternatif
4005,9005,snopes,Photographs show a real trampoline bridge in P...,2016-03-03,snopes,https://www.snopes.com/fact-check/trampoline-b...,Photographs purportedly showing a “trampoline ...,"[{""id"" : 17727"",""""begin"": 86,""end"": 98,""entity...",[],[],http://worldarchitecture.org/blogs/wanews/pzvc...,"bouncing bridge, bridge, trampoline bridge",FALSE: Photos of a ‘Trampoline Bridge’ Don’t S...,False
503,5503,snopes,A photograph shows a record-breaking 320-pound...,2017-04-14,snopes,https://www.snopes.com/fact-check/canadian-fis...,A photograph purportedly showing a giant 320-p...,"[{""id"" : 52288758"",""""begin"": 77,""end"": 94,""ent...",[],"[{""id"" : 47335"",""""begin"": 0,""end"": 7,""entity"":...","https://archive.is/0VXEc,https://www.snopes.co...","catfish, fake news, fishing, shrimp, world new...",Did Canadian Fishermen Catch a 320-Pound Shrimp?,False
999,5999,snopes,President Trump signed an Executive Order bann...,2017-02-07,snopes,https://www.snopes.com/fact-check/trump-eo-vac...,"On 6 February 2017, the Houston Leader web sit...","[{""id"" : 58911"",""""begin"": 388,""end"": 395,""enti...",[],[],"http://archive.is/GxMkN/image,https://twitter....","executive order, houston leader, president tru...",Did President Trump Enact a 90-Day Ban on Chil...,False
315,315,snopes,"Connor Betts, the suspect in the Dayton mass s...",2019-08-05,snopes,https://www.snopes.com/fact-check/dayton-shoot...,"On Aug. 4, 2019, a gunman opened fire in a cro...","[{""id"" : 8253"",""""begin"": 48,""end"": 59,""entity""...","[{""id"" : 31883778"",""""begin"": 39,""end"": 52,""ent...",[],https://www.nytimes.com/2019/08/04/us/dayton-o...,NaN,Was Dayton Mass Shooter a Self-Described ‘Pro-...,True
4476,14476,snopes,Letter from U.S. Army lieutenant serving in Ir...,2006-07-07,snopes,https://www.snopes.com/fact-check/tom-cotton/,Claim: Letter from U.S. Army lieutenant serv...,"[{""id"" : 7515928"",""""begin"": 48,""end"": 52,""enti...","[{""id"" : 7515928"",""""begin"": 42,""end"": 46,""enti...",[],"http://powerlineblog.com/archives/014515.asp,h...",ASP Article,Tom Cotton Letter,True
1420,11420,snopes,Actress Kristen Stewart trashed the military a...,2014-12-10,snopes,https://www.snopes.com/fact-check/red-white-an...,Claim: Actress Kristen Stewart trashed the m...,"[{""id"" : 2629332"",""""begin"": 14,""end"": 29,""enti...","[{""id"" : 2629332"",""""begin"": 8,""end"": 23,""entit...",[],http://www.thedailybeast.com/articles/2014/10/...,"Actors, ASP Article, Movies",Did Kristen Stewart Trash the Military and Def...,FALSE
2362,12362,snopes,A man left entertaining his girlfriend’s dog t...,2014-04-27,snopes,https://www.snopes.com/fact-check/rain-of-terr...,Claim: A man left entertaining his girlfrien...,"[{""id"" : 2018731"",""""begin"": 6,""end"": 11,""entit...","[{""id"" : 2018731"",""""begin"": 0,""end"": 5,""entity...",[],"/love/dating/sink.asp,http://www.imdb.com/titl...","ASP Article, Natural Phenomena",Dog Jumps Out Apartment Window to Chase Tennis...,LEGEND
2368,17368,snopes,Italian dictator Benito Mussolini made the tra...,2007-09-29,snopes,https://www.snopes.com/fact-check/loco-motive/,Claim: Italian dictator Benito Mussolini mad...,"[{""id"" : 19283178"",""""begin"": 23,""end"": 39,""ent...","[{""id"" : 19283178"",""""begin"": 17,""end"": 33,""ent...",[],NaN,"ASP Article, Government",Mussolini and On Time Trains,False
4865,4865,snopes,Despite arguing that government benefits const...,2017-06-23,snopes,https://www.snopes.com/fact-check/ayn-rand-soc...,"Russian-born American author Ayn Rand, who is ...","[{""id"" : 339"",""""begin"": 28,""end"": 36,""entity"":...","[{""id"" : 21554936"",""""begin"": 63,""end"": 87,""ent...","[{""id"" : 339"",""""begin"": 0,""end"": 8,""entity"": ""...",http://aynrandlexicon.com/lexicon/collectivism...,"ayn rand, objecti

In [34]:
vloc = (df.loc[503,"extra_mot_cle"])
pd.options.display.max_colwidth = 300
print(vloc)

503                                                                                                                                                                                                                                                                                                             []
503    [{"id" : 47335",""begin": 0,"end": 7,"entity": "Catfish","text": "catfish","score": 0.32,"categories" : ["Edible fish","Commercial fish","Siluriformes","Ostariophysi","Fish of Africa","Fish of Asia","Fish of Europe","Fish of South America","Fish of North America","Fish of Central America","Soul ...
503    [{"id" : 843491",""begin": 12,"end": 36,"entity": "Not Necessarily the News","text": "Not Necessarily The News","score": 0.50,"categories" : ["HBO network shows","American sketch comedy television shows","Satirical television programmes","Television series about television","1980s American telev...
Name: extra_mot_cle, dtype: object
